The following tutorial is how to extract features using the acoustic feature extraction library. The tutorial will also document places where the library can be improved upon.

The only dependency that requires installment is [librosa](https://github.com/librosa/librosa), an open-source acoustic and music feature extraction library.

In [ ]:
%pylab
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import os
import tables
import json
import music_feats.features.features as ft
import numpy as np

print "done importing"

In [ ]:
def nextPow(x):
    """Round input x to next highest power of 2."""
    return ceil(math.log(x, 2))
def prevPow(x):
    """Round input x to next smallest power of 2."""
    return floor(math.log(x, 2))

In [ ]:
# specify which sound file to use
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'tests', 'data'))

fname = "Beethoven_Op027No1-01_003_20090916-SMD-norm.wav" # sample wav file
trname = 'sess_1_block_0_AN.report' # sample report file

# some parameters for the feature extraction object. see docstring for full details
sr = 44100 # sampling rate
TR = 2.0045 # fMRI scanning sequence TR value

# creatue features.Features object corresponding to that audio file
feature_gen = ft.Features(os.path.join(data_dir, fname), TR=TR, sr=sr)

#### Parameters to set

In [ ]:
# these are booleans used to indicate if window/hop (overlap) lengths used during feature extraction
# should be rounded to the next highest or the next smallest power of 2
use_prev = False
use_next = False

# the window and overlap amount to be used with short duration feature
short_nfft = 2048 # ~50 ms granularity
short_hop = int(short_nfft / 2) # ~25 ms overlap between windows

# the window and overlap amount to be used with long duration features
seconds = 1.34 # TODO: with later versions of librosa try adjusting this parameter (see below)
long_nfft = int(seconds * sr) # convert the seconds to samples
if use_prev:
    long_nfft = 2**ft.prevPow(long_nfft) # ~0.743 sec
elif use_next:
    long_nfft = 2**ft.nextPow(long_nfft)
long_hop = int(long_nfft / 2) # half overlap

# binning spectrogram --> to be used with some of the long duration features
seconds = 4 # rounded down to nearest power of 2 (w/ respect to sr=44100)

# the intermediate step for FP and MPS
FP_nfft = 512 # number from Pampalks PhD Thesis (2006)
FP_hop = 512 # number from Pampalks PhD Thesis (2006); no overlap

# CQT parameters
cqt_hop = 1024
cqt_seconds = 2.0 # chunk size prior to CQT; need to chunk
n_bins = 120 # to match the tonotopic range
bins_per_octave = 12 # to match the tonotopic range; 12 bands per octave
fmin = 20 # minimum frequency used
use_han = False # don't use the hanning window because librosa seems to window it anyway when creating frames

### short and long duration acoustic features

Short and long duration features are based on [Alluri 2012](http://www.ncbi.nlm.nih.gov/pubmed/22116038) paper and the music information retrieval references cited within that paper

In [ ]:
# extract short duraction features
# check docstrings for full list of arguments; will use the default values for now

feature_gen.rms(n_fft=short_nfft, hop_length=short_hop)
feature_gen.temporalFlatness(hop_length=short_hop)
feature_gen.zcr(n_fft=short_nfft, hop_length=short_hop)
feature_gen.spectralCentroid(n_fft=short_nfft, hop_length=short_hop)
feature_gen.spectralSpread(n_fft=short_nfft, hop_length=short_hop)
feature_gen.spectralFlatness(hop_length=short_hop)
feature_gen.spectralContrast(n_fft=short_nfft, hop_length=short_hop)
feature_gen.spectralRolloff(n_fft=short_nfft, hop_length=short_hop)
feature_gen.mfcc(n_fft=short_nfft, hop_length=short_hop)
feature_gen.melspectrogram(n_fft=short_nfft, hop_length=short_hop)
feature_gen.stft(n_fft=short_nfft, hop_length=short_hop)
feature_gen.CQT(cqt_hop=cqt_hop, seconds=cqt_seconds, n_bins=n_bins,
                bins_per_octave=bins_per_octave, fmin=fmin, use_han=use_han)

print 'done extracting short length features'

##### Librosa v0.4.0 was used to develop the feature extraction library. At the time, long duration features were limited to be extracted over a 1.34 second window (even though the ideal window binning size might actually be 2-4ms according to the literature. This was a limitation in librosa's implementation of stft and should be explored with future versions of librosa. An alternative solution might be implementing one's own stft.

In [ ]:
# extract long duration features
# check docstrings for full list of arguments; will use the default values for now

feature_gen.chromagram(stft=True, n_fft=long_nfft, hop_length=long_hop, seconds=seconds, use_librosa=True)
feature_gen.tonalCentroid(chroma=feature_gen.returnFeature('chroma'))
feature_gen.tonality(n_fft=long_nfft, hop_length=long_hop, seconds=seconds, use_librosa=True)
feature_gen.fluctuationPatterns(n_fft=FP_nfft, hop_length=FP_hop, seconds=seconds)
feature_gen.fluctuationCentroid(n_fft=FP_nfft, hop_length=FP_hop, seconds=seconds)
feature_gen.fluctuationFocus(n_fft=FP_nfft, hop_length=FP_hop, seconds=seconds)
feature_gen.fluctuationEntropy(n_fft=FP_nfft, hop_length=FP_hop, seconds=seconds)

print 'done extracting long length features'

#### Print all features extracted

In [ ]:
print feature_gen.extractedFeatures()

## Some feature visualizations
### RMS

In [ ]:
plt.figure()
plt.plot(np.squeeze(feature_gen.returnFeature('RMS')))
plt.title('RMS')
plt.xlabel('time')

### Spectrogram (log scaled amplitude)

In [ ]:
dims = feature_gen.returnFeatureDim('S')
print dims

In [ ]:
plt.figure()
# flipping to have low frequencies at bottom and high frequencies at top
plt.imshow(np.flipud(feature_gen.returnFeature('S')), aspect='auto')
plt.xlabel('Time (window number)')
plt.ylabel('Frequency (Hz)')
plt.yticks(np.r_[0:1025:200][::-1], np.linspace(0, 22050, num=6))
plt.title('Spectrogram (dB)')
plt.show()

### [MFCC](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)

This feature is primarily relevant for vocal sound elements and not particularly informative for music and other non-vocal acoustic waveforms.

In [ ]:
plt.figure()
plt.imshow(np.flipud(feature_gen.returnFeature('MFCC')), aspect='auto')
plt.xlabel('time')
plt.yticks([0,19][::-1], ['Low coeff', 'High coeff'])
plt.title('Mel-frequency cepstral coefficients')
plt.show()

### Fluctuation patterns (rhythmic patterns)

Fluctuation patterns implementation based on:
+ [Pampalk Thesis, 2006](http://www.ofai.at/~elias.pampalk/publications/pampalk06thesis.pdf)
+ [Content-based Visualization and Organization of Music Archives](http://www.ifs.tuwien.ac.at/ifs/research/pub_pdf/pam_acmmm02.pdf)
+ [Islands of Music: Analysis, Organization, and Visualization of Music Archives](http://www.ofai.at/~elias.pampalk/music/)
+ [Fluctuation strength and temporal masking patterns of amplitude-modulated broadband noise](http://www.ncbi.nlm.nih.gov/pubmed/7142033)

Demo parameters are that we used 12 mel bands in the initial time-by-frequency representation, before computing the fluctuation patterns per frequency band. The maximum amplitude modulation frequency is 10Hz (also a parameter value that can be adjusted).

In [ ]:
dims = feature_gen.returnFeature('FP').shape
FP = feature_gen.returnFeature('FP')
print dims

The figure representation of the fluctuation patterns is a bit confusing at initial glance, but the break down is as such. The y-axis show the amplitude modulation frequencies for each of the frequency subbands that we took the fourier transform of. That is, for this example, each i-th continguous chunk of 360/12 (i.e, FP.shape[0]/num_mel_bands) corresponds to the fourier transform of i-th mel frequency subband of the melspectrogram, from which the fluctuation patterns are computed. The x-axis correponds to time segments that are 4 seconds long (reference **seconds** parameter defined above).

In [ ]:
plt.figure(figsize=(12,12))
# flipping to have low frequencies at bottom and high frequencies at top
plt.imshow(np.flipud(FP), aspect='auto', cmap='gray')
plt.xlabel('Time segment number')
plt.ylabel('Amplitude modulation per mel frequency band (Hz)')
plt.yticks(np.r_[0:dims[0]:dims[0]/12][::-1], np.arange(1,13))
plt.title('Fluctuation Patterns')
plt.show()

In the figure above, the mel frequency bands are enumerated. Below we visualize the fluctuation patterns correponding to one time segment (the first time segment). Follow the transformations done to have a better idea of what the representation is.

In [ ]:
# reshaping flunctuations for one time segment into the approrpriate size
single_time = FP[:,0]
print single_time.shape
FP_single_time = single_time.reshape((12, dims[0]/12))
print FP_single_time.shape

In [ ]:
plt.figure()
plt.imshow(np.flipud(FP_single_time.T), aspect='auto', cmap='gray')
plt.xlabel('Mel-Frequency Band')
plt.ylabel('Modulation Frequency (Hz)')
plt.yticks(np.r_[0:30:5], list(np.linspace(5, 10, num=6))[::-1])
plt.title('Fluctuation patterns for one time point')
plt.show()

### Tonality

References used for implementation:
+ [Tonal Description of Music Audio Signals](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.76.8192&rep=rep1&type=pdf)
+ [Tonal Description of Polyphonic Audio for Music Content Processing](http://www.mtg.upf.edu/node/457)
+ [Cognitive Foundations of Musical Pitch: A key-finding algorithm based on tonal hierarchies](http://www.utsc.utoronto.ca/~marksch/psyc56/Krumhansl%201990%20Ch%204.pdf)
+ [Cognitive Foundations of Musical Pitch: Quantifying tonal hierarchies and key distances](http://www.oxfordscholarship.com/view/10.1093/acprof:oso/9780195148367.001.0001/acprof-9780195148367-chapter-2)
+ [What's Key for Key? The Krumhansl-Schmuckler Key-Finding Algorithm Reconsidered](http://www.jstor.org/stable/40285812?seq=1#page_scan_tab_contents)

In [ ]:
keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
keyTS = feature_gen.returnFeature('keyTimeSeries').squeeze()
modeTS = feature_gen.returnFeature('modeTimeSeries').squeeze()

In [ ]:
keyMajor = np.ones(len(keyTS)) * -1
keyMajor[modeTS == 0] = keyTS[modeTS == 0]

keyMinor = np.ones(len(keyTS)) * -1
keyMinor[modeTS == 1] = keyTS[modeTS == 1]

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(keyMajor, '+', label='major')
plt.plot(keyMinor, '+', label='minor')
plt.legend()
plt.ylim([-0.25, 11.25])
plt.yticks(np.arange(12), np.array(keys))
plt.title('Tonality w/ modality')
plt.show()

### CQT

Even though the library relies upon librosa's implementation of the constant-q transform, the following is the literature that explains the theory and implementation.

+ [Calculation of the constant-q spectral transform](https://www.ee.columbia.edu/~dpwe/papers/Brown91-cqt.pdf)
+ [An efficient algorithm for the calculation of a constant Q transform](http://academics.wellesley.edu/Physics/brown/pubs/effalgV92P2698-P2701.pdf)
+ [The Constant Q Transform](http://doc.ml.tu-berlin.de/bbci/material/publications/Bla_constQ.pdf)
+ [Constant-Q Transform Toolbox for Music Processing](http://iem.kug.ac.at/fileadmin/media/iem/projects/2010/smc10_schoerkhuber.pdf)

In [ ]:
plt.figure()
# flipping to have low frequencies at bottom and high frequencies at top
plt.imshow(np.flipud(feature_gen.returnFeature('CQT')), aspect='auto')
plt.xlabel('Time (window number)')
plt.ylabel('Constant-Q Frequency Bands')
plt.yticks(np.r_[0:120:20][::-1], np.linspace(20, 120, num=6, endpoint=True))
plt.title('Constant-Q Transform (dB)')
plt.show()

## Saving extracted features

Saving has been left to use preference. Thus, the feature extraction object returns dictionaries containing the extracted features, the parameters used to extract these features, and the dimensions of each of the extracted features.

In [ ]:
all_features, featureParams, featureDims = feature_gen.saveFeatures()

In [ ]:
print featureParams['RMS']
print featureParams['CQT']

Save the features

In [ ]:
# hf = tables.open_file('sampleFeatures.hdf', mode='w', title='safe_file')
# for vname, var in all_features.items():
#     hf.create_array('/', vname, var)
# hf.close()

Save the featue params

In [ ]:
# with open('featureParams.json', 'w') as data_file:
#     json.dump(featureParams, data_file)

## Future directions and other remarks

It is highly encouraged to use the docstrings provided in the library to your advantage. This is only a brief survey of the parameters and features that are available for manipulation.

The library is currently v1, meaning that it is still a work in progress. Feel free to add to the library's functionality and/or address some of the TODO's in the source code (or even addressing the deprecation warnings above-during the feature extraction).

Have fun!